# Problem Setup
The user repositions a sparse set of points  and the goal is to propagate the transformations at these "handles" to the rest of the shape. 

Given the point rest position $\tilde x \in \mathbb R^3$, we will write its unknown deformed position as $x$ and the point's displacement vector $d := x - \tilde x$. 

The problem can be thought of two ways 
 - as a scattered data interpolation, where the handles are the sparse samples and we want to interpolate over the unknown displacement field
 - as a shape optimization problem, where we try to define a new shape that retains the details of the old shape but fulfill the handle constraints. 
 
## Properties to Consider In Deformation

### Continuity
The deformation should be continuous, i.e. the shape will not tear, crack or change topological features. i.e. the face matrix $F = \tilde F$ will not change, only $V$ should be determined from $\tilde V$. 

### Generic Distortion Minimization
A rest surface $\tilde S$ immersed in $\mathbb R^3$ can be described as a mapping $\tilde x$ from some 2D parametric domain $\Omega$. i.e. $\tilde x: \Omega \subseteq \mathbb R^2 \rightarrow \mathbb R^3$, similarly define the deformed surface as $x:\Omega \rightarrow \mathbb R^3$ and displacement vector field as $d(u, v) = x(u, v) - \tilde x(u, v)$. 

Consider distortion minimization and the handle's position as energy minimization with constraints. Suppose we have some energy functional $E(x)$ that measures the amount of distortion. A natural choice of distortion measurement is by intergrating local distortion over all points, i.e. we have the problem
\begin{align*}
\text{minimize} \quad &E(x) = \int_\Omega \|e(x)\|^2 dA\\
\text{subject to} \quad &\{x:\Omega \rightarrow \mathbb R^3\mid\forall i = \{1,...,k\}.  x(u_i, v_i) = g_i\}
\end{align*}
where $e$ is the measurement of local distortion, the function is upon our choice. 

# Gradient based Energy
If we assume that the deformation is small then we can measure the distortion as the magnitude of gradient of the displacement field, i.e. if the displacement field has large variations or sudden changes then it induces a lot of distortion. 

$$\int_\Omega \|e(x)\|^2 dA = \int_\Omega \|\mathbf Dd\|_F^2 dA $$
where $\mathbf Dd$ is the Jacobian of $d$.  

The discretized version over our triangle mesh gives 
$$tr(D^TLD)\text{ w.r.t } D_{handles} = g_{handles} - \tilde V_{handles}$$

However this methods is not smooth at constraints and the influence of handles dies off too quickly. By minimizing the Dirichlet energy, each coordinate of "displacement field" is a harmonic function. Intuitively (however abstractly) we can think of each function as diffusing the user's constraints as if they were heat values. As such, each function diffuses quickly to an average, slowly varying value over most of the domain. As a displacement field a constant value for each coordinate function would mean a translation: most of the shape is simply translated.

Note that Jacobian is a linear operator, i.e. 
$$\min_d\int \|\mathbf D d\|_F^2 dA = \min_x \int \| \mathbf D x - \mathbf D\tilde x\|_F^2 dA$$
If we think of the gradient of the position function  (with respect to the underlying parameterization) as a local geometric feature descriptor then this energy can be re-understood as measuring the difference in this feature before and after the deformation. This is very sensible as we are trying to measure distortion. We would expect that a low-distortion deformation would correspond with a small change to local features.


# Laplacian based Energy

If we model distortion as the change in a local feature descriptor, then a natural local and relative descriptor would be one that compared the position of some point on the shape to the average of its local neighborhood. Consider the Laplace operator as taking exactly the difference of a function value at a point and the average over an infinitesimal region 
$$\Delta f (x) = \big[\lim_{|B_\delta(x)|\rightarrow 0} B_\delta(x)^{-1}\int_B f(z)dz\big] - f(x)$$
When applied to the embedding function $x$ the Laplace operator computes the differences in position between a point and its local neighborhood. The vector points in the normal direction and its magnitude corresponds inversely with how flat the surface is locally. 

We can use calculus of variations apply Green's identity twice to show that minimizers of the squared Laplacian energy are _bi-harmonic_ functions ($\Delta^2 d = 0$). This will allow us to ensure continuity of first derivatives across constrained values at handles. 

## Discretized Approximation
(For more details of the discretization and symbols, go to smoothing)

We can make sure our discrete Laplacian operator $L$. This matrix computes the locally integrated Laplacian of a given function specified by per=vertex values $\mathbf f$. We can approximate the point-wise Laplacian by the local integral average of the Laplaicna $M^{-1}L\mathbf f$. Integrating this over the mesh we have the complete approximation
$$\int_\Omega \|\Delta d\|^2 dA \approx tr(D^TLM^TMM^{-1}LD) = tr(D^T\underset{Q}{L^TM^{-1}L}D)$$
where $M$ is the mass-matrix for the given mesh and $Q = L^TM^{-1}L$ can be thought of as the bi-Laplacian matrix.

## Precomputation
Without loss of generality, assume that the rows of the unknown displacement $D$ have been sorted so that displacements corresponding to handle vertices are in the bottom part, i.e. $D = \begin{pmatrix}D_u\\D_{handle}\end{pmatrix}$

Since the displacement at handles are, we can separate and eliminate it from optimization
\begin{align*}
\min_{D_u}tr(D^T Q D) &= \min_{D_u} tr\big(\begin{bmatrix}D^T_u&D_h^T\end{bmatrix}
\begin{bmatrix}Q_{u,u} & Q_{u,h}\\
Q_{h,u} &Q_{h,h}\end{bmatrix}
\begin{bmatrix}D_u\\D_{h}\end{bmatrix}\big)\\
&= \min_{D_u} tr\big(D^TQ_{u,u} D_u + 2D^T_u Q_{u,h}D_h\big)\\
&= \min_{D_u} tr(E(D_u))
\end{align*}
Then, set $\partial_{D_u}E = 0$ for the solution of the quadratic optimization
\begin{align*}
\partial_{D_u} E &= 2Q_{u,u} D_u + 2Q_{u,h} D_h\\
\Rightarrow D_u &= Q^{-1}_{u,u} Q_{u,h} D_h
\end{align*}
Since $Q_{u,u}$ does not change, we can prefactorize it so that applying its inverse is fast. 

## Problem

Biharmonic displacements works well for small deformations and deformations that do not imply a large rotation. The Laplacian of the position function $\Delta x$ as a feature descriptor is not rotation invariant. This problem is also true for all linear differential features. This means that if the user transforms all the handle locations by a rigid transformation $T$ these energies will measure zero. We would like global rotation invariant, but we should also like this property to apply locally so that parts of the shape can easily rotate. 

# As Rigid As Possible